## Operations Reserach Club
## Optimization with Google OR tools

Please visit the following website for the tutorials and documentation. 

https://developers.google.com/optimization/introduction/python

We will maximize the usage of the operations room of a hospital. It has multiple op-rooms and multiple departments that are doing the operations in those op-rooms.

In [2]:
from ortools.linear_solver import pywraplp

In [3]:
def create_data_model():
    """Create the data for the example."""
    data = {}
    weights = [48, 30, 42, 36, 36, 48, 42, 42, 36, 24, 30, 30, 42, 36, 36]
    values = [10, 30, 25, 50, 35, 30, 15, 40, 30, 35, 45, 10, 20, 30, 25]
    data['weights'] = weights
    data['values'] = values
    data['items'] = list(range(len(weights)))
    data['num_items'] = len(weights)
    
    num_bins = 5
    data['bins'] = list(range(num_bins))
    data['bin_capacities'] = [100, 100, 100, 100, 100]
    
    num_deps = 2
    data['deps'] = list(range(num_deps))
    data['dep_capacities'] = [100, 100]
    
    return data

In [4]:
# Create the mip solver with the SCIP backend.
solver = pywraplp.Solver.CreateSolver('SCIP')

In [5]:
data=create_data_model()
data

{'weights': [48, 30, 42, 36, 36, 48, 42, 42, 36, 24, 30, 30, 42, 36, 36],
 'values': [10, 30, 25, 50, 35, 30, 15, 40, 30, 35, 45, 10, 20, 30, 25],
 'items': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14],
 'num_items': 15,
 'bins': [0, 1, 2, 3, 4],
 'bin_capacities': [100, 100, 100, 100, 100],
 'deps': [0, 1],
 'dep_capacities': [100, 100]}

In [6]:
# Variables
# x[i, j] = 1 if item i is packed in bin j.
x = {}
for i in data['items']:
    for j in data['bins']:
        for k in data['deps']:
            x[(i,j,k)] = solver.IntVar(0, 1, 'x_%i_%i_%i' % (i,j,k))

In [7]:
x

{(0, 0, 0): x_0_0_0,
 (0, 0, 1): x_0_0_1,
 (0, 1, 0): x_0_1_0,
 (0, 1, 1): x_0_1_1,
 (0, 2, 0): x_0_2_0,
 (0, 2, 1): x_0_2_1,
 (0, 3, 0): x_0_3_0,
 (0, 3, 1): x_0_3_1,
 (0, 4, 0): x_0_4_0,
 (0, 4, 1): x_0_4_1,
 (1, 0, 0): x_1_0_0,
 (1, 0, 1): x_1_0_1,
 (1, 1, 0): x_1_1_0,
 (1, 1, 1): x_1_1_1,
 (1, 2, 0): x_1_2_0,
 (1, 2, 1): x_1_2_1,
 (1, 3, 0): x_1_3_0,
 (1, 3, 1): x_1_3_1,
 (1, 4, 0): x_1_4_0,
 (1, 4, 1): x_1_4_1,
 (2, 0, 0): x_2_0_0,
 (2, 0, 1): x_2_0_1,
 (2, 1, 0): x_2_1_0,
 (2, 1, 1): x_2_1_1,
 (2, 2, 0): x_2_2_0,
 (2, 2, 1): x_2_2_1,
 (2, 3, 0): x_2_3_0,
 (2, 3, 1): x_2_3_1,
 (2, 4, 0): x_2_4_0,
 (2, 4, 1): x_2_4_1,
 (3, 0, 0): x_3_0_0,
 (3, 0, 1): x_3_0_1,
 (3, 1, 0): x_3_1_0,
 (3, 1, 1): x_3_1_1,
 (3, 2, 0): x_3_2_0,
 (3, 2, 1): x_3_2_1,
 (3, 3, 0): x_3_3_0,
 (3, 3, 1): x_3_3_1,
 (3, 4, 0): x_3_4_0,
 (3, 4, 1): x_3_4_1,
 (4, 0, 0): x_4_0_0,
 (4, 0, 1): x_4_0_1,
 (4, 1, 0): x_4_1_0,
 (4, 1, 1): x_4_1_1,
 (4, 2, 0): x_4_2_0,
 (4, 2, 1): x_4_2_1,
 (4, 3, 0): x_4_3_0,
 (4, 3, 1): x

In [8]:
# Constraints
# Each item can be in at most one bin.
for i in data['items']:
    solver.Add(sum(x[i, j,k] for j in data['bins'] for k in data['deps']) <= 1)
            
        
            
# The amount packed in each bin cannot exceed its capacity.
for j in data['bins']:
    solver.Add(
                sum(x[(i, j,k)] * data['weights'][i]
                    for i in data['items'] for k in data['deps']) <= data['bin_capacities'][j])
    
# The amount packed in each department cannot exceed its capacity.
for k in data['deps']:
    solver.Add(
                sum(x[(i, j,k)] * data['weights'][i]
                    for i in data['items'] for j in data['bins']) <= data['dep_capacities'][k])

In [9]:
print('Number of constraints =', solver.NumConstraints())

Number of constraints = 22


In [10]:
# Objective
objective = solver.Objective()

for i in data['items']:
    for j in data['bins']:
        for k in data['deps']:
            objective.SetCoefficient(x[(i, j,k)], data['values'][i])
objective.SetMaximization()

In [11]:
status = solver.Solve()
if status == pywraplp.Solver.OPTIMAL:
    print('Total packed value:', objective.Value())
    total_weight = 0
    for j in data['bins']:
        bin_weight = 0
        bin_value = 0
        print('Bin ', j, '\n')
        for i in data['items']:
            for k in data['deps']:
                if x[i, j,k].solution_value() > 0:
                    print('Item', i, '- weight:', data['weights'][i], ' value:',data['values'][i])
                    bin_weight += data['weights'][i]
                    bin_value += data['values'][i]
        print('Packed bin weight:', bin_weight)
        print('Packed bin value:', bin_value)
        print()
        total_weight += bin_weight
    print('Total packed weight:', total_weight)
else:
    print('The problem does not have an optimal solution.')

Total packed value: 225.0
Bin  0 

Item 1 - weight: 30  value: 30
Item 10 - weight: 30  value: 45
Packed bin weight: 60
Packed bin value: 75

Bin  1 

Item 13 - weight: 36  value: 30
Packed bin weight: 36
Packed bin value: 30

Bin  2 

Item 3 - weight: 36  value: 50
Item 4 - weight: 36  value: 35
Packed bin weight: 72
Packed bin value: 85

Bin  3 

Packed bin weight: 0
Packed bin value: 0

Bin  4 

Item 9 - weight: 24  value: 35
Packed bin weight: 24
Packed bin value: 35

Total packed weight: 192


In [12]:
for i,j in x.items():
    print('item ', i[0], ' is in the department ', i[2],'and in OP-room ', i[1], j.solution_value())

item  0  is in the department  0 and in OP-room  0 0.0
item  0  is in the department  1 and in OP-room  0 0.0
item  0  is in the department  0 and in OP-room  1 0.0
item  0  is in the department  1 and in OP-room  1 0.0
item  0  is in the department  0 and in OP-room  2 0.0
item  0  is in the department  1 and in OP-room  2 0.0
item  0  is in the department  0 and in OP-room  3 0.0
item  0  is in the department  1 and in OP-room  3 0.0
item  0  is in the department  0 and in OP-room  4 0.0
item  0  is in the department  1 and in OP-room  4 0.0
item  1  is in the department  0 and in OP-room  0 1.0
item  1  is in the department  1 and in OP-room  0 0.0
item  1  is in the department  0 and in OP-room  1 0.0
item  1  is in the department  1 and in OP-room  1 0.0
item  1  is in the department  0 and in OP-room  2 0.0
item  1  is in the department  1 and in OP-room  2 0.0
item  1  is in the department  0 and in OP-room  3 0.0
item  1  is in the department  1 and in OP-room  3 0.0
item  1  i